<a href="https://colab.research.google.com/github/ishanmorwal/Rock-Paper-Scissors/blob/main/RockPaperScissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement
We all know and enjoy Stone,Paper,Scissors. It is a very popular game and rightfully so, for its simplicity yet enjoyability is off the charts. This model here, is an image classification model to classify an image as either the image of a stone or a paper or a scissors in the Sign Language.

In [24]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt

After the imports, we download the dataset and unzip it.

In [25]:
# prompt: download and unzip the kaggle stone-paper-scissors dataset
!wget --no-check-certificate \
"https://storage.googleapis.com/download.tensorflow.org/data/rps.zip"\
-O "/tmp/Rock-paper-scissors.zip"
lzip = '/tmp/Rock-paper-scissors.zip'
zip_ref = zipfile.ZipFile(lzip,'r')
zip_ref.extractall('/tmp')
zip_ref.close()
!ls /tmp/rps

--2024-02-09 07:35:10--  https://storage.googleapis.com/download.tensorflow.org/data/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.137.207, 142.250.141.207, 142.251.2.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.137.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200682221 (191M) [application/zip]
Saving to: ‘/tmp/Rock-paper-scissors.zip’

/tmp/Rock-paper-sci 100%[===================>] 191.38M   211MB/s    in 0.9s    

2024-02-09 07:35:11 (211 MB/s) - ‘/tmp/Rock-paper-scissors.zip’ saved [200682221/200682221]

paper  rock  scissors  training  validation


The dataset is now downloaded and stored within the /tmp directory and is extracted. Within it, there

In [26]:
src_path = '/tmp/rps'

src_path_rock     = os.path.join(src_path,'rock')
src_path_paper    = os.path.join(src_path,'paper')
src_path_scissors = os.path.join(src_path,'scissors')


# Prints out the number of images in the sub-directories
print(f"There are {len(os.listdir(src_path_rock))} images of Rock in sign language.")
print(f"There are {len(os.listdir(src_path_paper))} images of Paper in sign language.")
print(f"There are {len(os.listdir(src_path_scissors))} images of Scissors in sign language.")


There are 840 images of Rock in sign language.
There are 840 images of Paper in sign language.
There are 840 images of Scissors in sign language.


Now we divide the data contained into training and validation directories.

In [27]:
root_dir = '/tmp/rps'

# Makes sure directory is empty
if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

# This function takes the root as argument and creates the directories

def create_train_val_dirs(root_path):
  os.makedirs(os.path.join(root_path,'training'))
  os.makedirs(os.path.join(root_path,'validation'))
  os.makedirs(os.path.join(root_path,'training/rock'))
  os.makedirs(os.path.join(root_path,'training/paper'))
  os.makedirs(os.path.join(root_path,'training/scissors'))
  os.makedirs(os.path.join(root_path,'validation/rock'))
  os.makedirs(os.path.join(root_path,'validation/paper'))
  os.makedirs(os.path.join(root_path,'validation/scissors'))

try:
  create_train_val_dirs(root_path=root_dir)
except FileExistsError:
  print("Only happens if the root_dir is not empty, which should not be the case here.")

In [28]:
# For testing the above code:

for rootdir,dirs,files in os.walk(root_dir):
  for subdir in dirs:
    print(os.path.join(rootdir,subdir))

/tmp/rps/training
/tmp/rps/validation
/tmp/rps/training/scissors
/tmp/rps/training/paper
/tmp/rps/training/rock
/tmp/rps/validation/scissors
/tmp/rps/validation/paper
/tmp/rps/validation/rock


The next logical step here, is to split the data to populate the various directories in a random fashion. We do that using a `split_data` function, which takes the source directory, the training and validation directories as the arguments, along with a `split_size` as well, which determines the ratio of images used for training versus validation.

Plus, to maintain the correctness of our dataset, we need to put in place a check that ensures that there are no files that have zero-length.

In [29]:
def split_data(SOURCE_DIR,TRAINING_DIR,VALIDATION_DIR,SPLIT_SIZE):
  filtered_files = []

  for file_name in os.listdir(SOURCE_DIR):
    file_path = os.path.join(SOURCE_DIR,file_name)
    if os.path.getsize(file_name):
      filtered_files.appened(file_name)
    else:
      print(f"{file_name} is zero length, therefore not used")

  random_files = random.sample(filtered_files,len(filtered_files))
  split = int(SPLIT_SIZE*len(filtered_files))
  training_files = random_files[:split]
  validation_files = random_files[split:]

  for file_name in training_files:
    copyfile(os.path.join(SOURCE_DIR,file_name),os.path.join(TRAINING_DIR))

  for file_name in validation_files:
    copyfile(os.path.join(SOURCE_DIR,file_name),os.path.join(VALIDATION_DIR))

That was just the division of data into Training and Validation directories. Now, we set up the directories of the three classes, Rock, Paper and Scissors.

In [31]:
ROCK_SOURCE_DIR    = "/tmp/rps/rock"
PAPER_SOURCE_DIR   = "/tmp/rps/paper"
SCISSORS_SOURCE_DIR = "/tmp/rps/scissors"

TRAINING_DIR   = "/tmp/rps/training/"
VALIDATION_DIR = "/tmp/rps/validation/"

TRAINING_ROCK_DIR       = os.path.join(TRAINING_DIR,"rock/")
VALIDATION_ROCK_DIR     = os.path.join(VALIDATION_DIR,"rock/")
TRAINING_PAPER_DIR      = os.path.join(TRAINING_DIR,"paper/")
VALIDATION_PAPER_DIR    = os.path.join(VALIDATION_DIR,"paper/")
TRAINING_SCISSORS_DIR   = os.path.join(TRAINING_DIR,"scissors/")
VALIDATION_SCISSORS_DIR = os.path.join(VALIDATION_DIR,"scissors/")

# Ensures the directories are empty in case of multiple uses of this code

if len(os.listdir(TRAINING_ROCK_DIR)) > 0:
  for file in os.scandir(TRAINING_ROCK_DIR):
    os.remove(file.path)
if len(os.listdir(TRAINING_PAPER_DIR)) > 0:
  for file in os.scandir(TRAINING_PAPER_DIR):
    os.remove(file.path)
if len(os.listdir(TRAINING_SCISSORS_DIR)) > 0:
  for file in os.scandir(TRAINING_SCISSORS_DIR):
    os.remove(file.path)

if len(os.listdir(VALIDATION_ROCK_DIR))>0:
  for file in os.scandir(VALIDATION_ROCK_DIR):
    os.remove(file.path)
if len(os.listdir(VALIDATION_PAPER_DIR))>0:
  for file in os.scandir(VALIDATION_PAPER_DIR):
    os.remove(file.path)
if len(os.listdir(VALIDATION_SCISSORS_DIR))>0:
  for file in os.scandir(VALIDATION_SCISSORS_DIR):
    os.remove(file.path)

# Specify the split ratio
split_size = .9

# Running the function
split_data(ROCK_SOURCE_DIR,TRAINING_ROCK_DIR,VALIDATION_ROCK_DIR,split_size)
split_data(PAPER_SOURCE_DIR,TRAINING_PAPER_DIR,VALIDATION_PAPER_DIR,split_size)
split_data(SCISSORS_SOURCE_DIR,TRAINING_SCISSORS_DIR,VALIDATION_SCISSORS_DIR,split_size)

print(f"\nThere are {len(os.listdir(TRAINING_ROCK_DIR))} images of rock-sign for training")
print(f"There are {len(os.listdir(TRAINING_PAPER_DIR))} images of paper-sign for training")
print(f"There are {len(os.listdir(TRAINING_SCISSORS_DIR))} images of scissors-sign for training")
print(f"There are {len(os.listdir(VALIDATION_ROCK_DIR))} images of rock-sign for validation")
print(f"There are {len(os.listdir(VALIDATION_PAPER_DIR))} images of paper-sign for validation")
print(f"There are {len(os.listdir(VALIDATION_SCISSORS_DIR))} images of scissors-sign for validation")


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/rps/rock'